<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/data2day_2023_polars/blob/main/data2day_2023_Polars_Teil_2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Polars: Der Turbo Boost für Dataframes - Teil 2

Wichtige Links zur Erinnerung:

- Homepage von Polars: https://www.pola.rs/
- User-Guide: https://pola-rs.github.io/polars/user-guide/
- API-Referenz: https://pola-rs.github.io/polars/py-polars/html/reference/

## Installation + Vorbereitung

In [ ]:
import urllib.request
import os.path

In [ ]:
REQUIREMENTS_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/requirements.txt"
urllib.request.urlretrieve(REQUIREMENTS_URL, os.path.basename(REQUIREMENTS_URL))

In [ ]:
# nicht vergessen, dass die Laufzeitumgebung ggf. neu gestartet werden muss
!pip install -qr requirements.txt

In [ ]:
import polars as pl

In [ ]:
# bis zu 60 Zeichen pro Spalte ausgeben und Fließkommazahlen nicht abkürzen
pl.Config(fmt_str_lengths=60, fmt_float="full")

In [ ]:
# CSV Daten herunterladen
DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/spotify-charts-2017-2021-global-top200.csv.gz"
LOCAL_DATA_FILE_NAME = os.path.basename(DATA_URL)
urllib.request.urlretrieve(DATA_URL, LOCAL_DATA_FILE_NAME)

In [ ]:
# Übungen und Hilfsfunktionen herunterladen
EXERCISES_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/data2day_exercises.py"
urllib.request.urlretrieve(EXERCISES_URL, os.path.basename(EXERCISES_URL))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from data2day_exercises import *

In [ ]:
# Daten aus CSV-Datei einlesen und Datums-Spalten parsen
df = pl.read_csv("spotify-charts-2017-2021-global-top200.csv.gz", try_parse_dates=True)
df.head(2) # die ersten 2 Zeilen ausgeben

## Aggregationen auf Gruppen

Du hast im ersten Teil schon erste Aggregat-Funktionen wie `max`, `min`, `mean` und `sum` kennengelernt. Wirklich mächtig werden diese Funktionen, wenn du sie auf Gruppen anwendest, die du aus fast beliebigen Ausdrücken bilden kannst.

Die Gruppe wird mit der Methode `group_by` gebildet.

Die anschließende Aggregation mit der Methode `agg`. Diese Methode funktioniert ähnlich wie ein `select`, aber für Aggregationen.

In [ ]:
# die fünf am meisten gestreamten Künstler
(df
    .group_by("artist")
    .agg(pl.col("streams").sum())
    .top_k(5, by="streams")
)

In einer Aggregation lassen sich auch mehrere Ausdrücke angeben...

In [ ]:
# die fünf am meisten gestreamten Künstler und ihre durchschnittliche Platzierung in den Charts
(df
    .group_by("artist")
    .agg(pl.col("streams").sum(), pl.col("rank").mean())
    .top_k(5, by="streams")
)

Die Gruppierung ist auch mit mehreren Ausdrücken möglich...

In [ ]:
# die 5 in einem Jahr am meisten gestreamten Künstler
(df
    .group_by("artist", pl.col("date").dt.year().alias("year"))
    .agg(pl.col("streams").sum())
    .top_k(5, by="streams")
    .sort("year")
)

Jetzt fehlt uns aber das Jahr 2020! Zum Glück funktioniert die Funktion `head` auch auf einem gruppierten Dataframe und liefert dann die ersten *n* Zeilen je Gruppe. Nachteil gegenüber `top_k`: wir müssen dafür den Datensatz vollständig sortieren.

In [ ]:
# Künstler mit den meisten Streams je Jahr
(df
    .group_by("artist", pl.col("date").dt.year().alias("year"))
    .agg(pl.col("streams").sum())
    .sort("streams", descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

Du kannst auch schauen, welche Künstler die meisten verschiedenen Songs in den Top-200 hatten.

In [ ]:
# Künstler mit den meisten verschiedenen Songs in den Top 200 je Jahr
(df
    .group_by("artist", pl.col("date").dt.year().alias("year"))
    .agg(pl.col("title").n_unique().alias("distinctSongsInTop200"))
    .sort("distinctSongsInTop200", descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

Wie würde die Rangliste aussehen, wenn du die Tage auf Nummer 1 als Maßstab nimmst?

In [ ]:
# Künstler mit den meisten Tagen an Nummer 1 je Jahr
(df
    .filter(pl.col("rank").eq(1))
    .group_by("artist", pl.col("date").dt.year().alias("year"))
    .agg(pl.count().alias("daysOnNumberOne"))
    .sort("daysOnNumberOne", descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

In [ ]:
# Anstatt den gesamten Datensatz zu filtern, können wir sogar Daten in der Aggregation filtern
(df
    .group_by("artist", pl.col("date").dt.year().alias("year"))
    .agg(
        pl.col("streams").sum(),
        pl.col("date").filter(pl.col("rank").eq(1)).count().alias("daysOnNumberOne")
    )
    .sort(["daysOnNumberOne", "streams"], descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

Wir haben mehr als einen Künstler pro Zeile, weil wir jede Kollaboration als einen eigenen Künstler bewerten. Es gibt viele bekannte Lieder aus solchen Kollaborationen...

In [ ]:
top_5_colabs = (df
    .filter(pl.col("artist").str.contains(", "))
    .group_by("artist", "title", "url")
    .agg(pl.col("streams").sum())
    .top_k(5, by="streams")
)
top_5_colabs

In [ ]:
play_song(top_5_colabs, 0)

## Listen als spezieller Datentyp

Polars kann auch sehr gut mit Listen als speziellem Datentyp umgehen. Eine solche Liste ensteht, wenn wir z.B. eine Zeichenkette mit der Methode `str.split` aufteilen.

In [ ]:
# "artist" als Zeichenkette
df.filter(pl.col("artist").eq("Shawn Mendes, Camila Cabello")).head(1)

In [ ]:
# "artist" als Liste von Zeichenketten
(df
    .filter(pl.col("artist").eq("Shawn Mendes, Camila Cabello"))
    .head(1)
    .with_columns(pl.col("artist").str.split(", "))
)

Es ist manchmal sehr nützlich solche Listen mit der Methode `explode` auszurollen. Dadurch wird der Datensatz dann entsprechend häufig dupliziert und kann wie jede andere Spalte auch behandelt werden.

In [ ]:
(df
    .filter(pl.col("artist").eq("Shawn Mendes, Camila Cabello"))
    .head(1)
    .with_columns(pl.col("artist").str.split(", "))
    .explode("artist")
)

Wir können jetzt die Künstler mit den meisten Tagen auf Nummer 1 berechnen ohne jede Kollaboration als eigenen Künstler zu interpretieren.

In [ ]:
# artists with most days on number 1 per year
(df
    .with_columns(pl.col("artist").str.split(", "))
    .explode("artist")
    .group_by("artist", pl.col("date").dt.year().alias("year"))
    .agg(
        pl.col("streams").sum(),
        pl.col("date").filter(pl.col("rank").eq(1)).count().alias("daysOnNumberOne")
    )
    .sort(["daysOnNumberOne", "streams"], descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

Anstatt die Listen auszurollen, können wir auch direkt auf Listen-Spalten arbeiten. Passende Methode sind im Kontext `list`, z.B. `list.lengths()` für die Länge einer Liste.

In [ ]:
# Wie viele Künstler sind je Top-200 Eintrag vorhanden?
(df
    .select(pl.col("artist"))
    .with_columns(pl.col("artist").str.split(", "))
    .with_columns(pl.col("artist").list.lengths().alias("artistCount"))
    .group_by("artistCount")
    .count()
    .sort("artistCount")
    .with_columns((pl.col("count")/pl.col("count").sum()).round(2).alias("percentage"))
)

In [ ]:
# Die Chart-Platzierung mit 10 Künstlern ist "Pa' La Cultura" auf Platz 151 am 7.8.2020
play_song(df
    .with_columns(pl.col("artist").str.split(", ").list.lengths().alias("artistCount"))
    .filter(pl.col("artistCount").eq(10))
)

## Übungen zu Gruppierungen und Aggregationen

### Frage 13

In [ ]:
q13.question()

In [ ]:
q13_df = ...

In [ ]:
q13.check(q13_df)
#q13.hint()
#q13.solution()

### Frage 14

In [ ]:
q14.question()

In [ ]:
q14_df = ...

In [ ]:
q14.check(q14_df)

### Frage 15

In [ ]:
q15.question()

In [ ]:
q15_df = ...

In [ ]:
q15_df

In [ ]:
q15.check(q15_df)

## Joins & Co. - Dataframes verbinden

### Verketten mit `pl.concat`
Eine flexible und einfache Art zwei Dataframes zu verbinden ist die Methode `pl.concat`.

In [ ]:
# how="vertical" stapelt zwei Dataframes übereinander, Namen und Typen der Spalten müssen übereinstimmen
pl.concat([
    df.sample(1),
    df.sample(1)
], how="vertical")

In [ ]:
# how="vertical_relaxed" versucht die Datentypen anzupassen, wenn notwendig
pl.concat([
    df.sample(1),
    df.sample(1).with_columns(pl.col("artist").cast(pl.Categorical))
], how="vertical_relaxed")

In [ ]:
# how="diagonal" kommt auch mit anderen Spalten-Namen klar
pl.concat([
    df.sample(1).select("title", "artist", pl.col("rank").alias("position")),
    df.sample(1).select("title", pl.col("artist").alias("performer"), "rank")
], how="diagonal")

In [ ]:
# how="horizontal" stapelt Dataframes nebeneinander, die Anzahl der Datensätze muss übereinstimmen
some_df = df.sample(4)
pl.concat([
    some_df.select("title", "artist"),
    some_df.select("streams", "rank")
], how="horizontal")

In [ ]:
# how="align" stapelt Dataframes nebeneinander und versucht sie an den gemeinsamen Schlüssel-Spalten auszurichten
pl.concat([
    some_df.sample(fraction=1.0, shuffle=True).select("url", "date", "title"),
    some_df.sample(fraction=1.0, shuffle=True).select("url", "date", "artist"),
    some_df.sample(fraction=0.5, shuffle=True).select("url", "date", "streams")
], how="align")

Mit `how=align` wird eigentlich schon ein Join durchgeführt, wobei nicht wirklich klar ist auf welchen Spalten.

In den meisten Fällen wird es deshalb besser sein, einen expliziten Join durchzuführen.

### Verbinden mit `join`

Mit Joins können wir zwei Dataframes verbinden. Polars unterstützt folgende Join-Typen:

`left.join(right, on=..., how=...)`

- `outer`: alle Zeilen aus `left` und `right`, auch wenn diese keinen Join-Partner im anderen Dataframe haben
- `left`: alle Zeilen aus `left`, auch wenn diese keinen Join-Partner in `right` haben
- `inner`: Zeilen aus `left` und `right` mit passendem Join-Partner im anderen Dataframe
- `semi`: Zeilen aus `left` mit passendem Join-Partner in `right` (wie `inner`, aber keine neuen Spalten von `right`)
- `anti`: Zeilen aus `left` ohne passendem Join-Partner in `right` (Gegenteil von `semi`)

In [ ]:
left = pl.DataFrame({
    "key": [0, 1, 2],
    "value": ["a", "b", "c"]
})
right = pl.DataFrame({
    "key": [1, 2, 3],
    "value": ["x", "y", "z"]
})

In [ ]:
# outer join
left.join(right, on="key", how="outer").sort("key")

In [ ]:
# left join
left.join(right, on="key", how="left")

In [ ]:
# inner join
left.join(right, on="key", how="inner")

In [ ]:
# semi join
left.join(right, on="key", how="semi")

In [ ]:
# anti join
left.join(right, on="key", how="anti")

## Übungen zu Joins

### Frage 16

In [ ]:
q16.question()

In [ ]:
q16_df = ...

In [ ]:
q16.check(q16_df)

### Frage 17

In [ ]:
q17.question()

In [ ]:
q17_df = ...

In [ ]:
q17.check(q17_df)

### Frage 18

In [ ]:
q18.question()

In [ ]:
q18_df = ...

In [ ]:
q18.check(q18_df)

## Gruppieren und Joinen mit Ausdrücken: `over`-Expressions

Für viele Berechnungen kann es hilfreich sein, einen Ausdruck über eine Gruppe auszuwerten.

Wir könnten z.B. versuchen den Newcomer des Jahres zu bestimmen. Dafür benötigen wir eine Information, wann ein Künstler das erste mal in den Charts auftauchte.

In [ ]:
first_appearance = df.group_by("artist").agg(pl.col("date").min().alias("firstChartAppearance"))
first_appearance.filter(pl.col("artist").is_in(["Billie Eilish", "Lewis Capaldi"]))

Diese neue Information können wir jetzt an den Gesamtdatensatz joinen und so den Newcomer des Jahres ermitteln.

In [ ]:
(df
    .join(first_appearance, on="artist")
    .filter(pl.col("date").dt.year().eq(pl.col("firstChartAppearance").dt.year()))
    .group_by(pl.col("date").dt.year().alias("year"), "artist")
    .agg(pl.col("streams").sum())
    .sort("streams", descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

Ed Sheeran war zwar kein Newcomer mehr im Jahr 2017, aber uns fehlt die Informtionen aus den Vorjahren, um das besser zu machen...

Wir können das gleiche aber auch ohne den Zwischendatensatz erreichen, in dem wir eine `over`-Expression nutzen.

In [ ]:
(df
    # Ausdruck mit over statt temporärer Dataframe mit group_by, agg und join
    .with_columns(pl.col("date").min().over("artist").alias("firstChartAppearance"))
    .filter(pl.col("date").dt.year().eq(pl.col("firstChartAppearance").dt.year()))
    .group_by(pl.col("date").dt.year().alias("year"), "artist")
    .agg(pl.col("streams").sum())
    .sort("streams", descending=True)
    .group_by("year")
    .head(1)
    .sort("year")
)

## Reshaping

Für manche Berechnungen und besonders auch Plots ist es hilfreich zwischen verschiedenen Varianten eines Dataframes zu wechseln.

Das Wide-Format hat mehr Spalten (Wide) und dafür weniger Zeilen.
Das Long-Format hat mehr Zeilen (Long) und dafür weniger Spalten.

In [ ]:
some_df = pl.DataFrame({
    "month": ["2023-01", "2023-01", "2023-01", "2023-02"],
    "genre": ["pop", "rock", "hip-hop", "pop"],
    "streams": [100, 200, 300, 150] 
})
some_df

Mit der Methode `pivot` können wir einen Datensatz *breiter* machen, also Informationen aus Zeilen in neue Spalten transportieren. Folgende Parameter sind wichtig:

- `index`: Spalten, die erhalten bleiben
- `columns`: Spalte mit Werten, aus denen neuen Spaltennamen gebildet werden
- `values`: Spalte mit Werten, die in die neuen Spalten geschrieben werden


In [ ]:
some_df.pivot(index="month", values="streams", columns="genre")

Die enstandenen `null` Werte könnten wir mit `fill_null` ersetzen.

In [ ]:
some_df.pivot(index="month", values="streams", columns="genre").fill_null(0)

Mit dem Gegenstück `melt` können wir einen Datensatz wieder länger machen, also Informationen aus Spalten in Zeilen transportieren. Folgende Parameter sind wichtig:

- `id_vars`: Spalten, die erhalten bleiben
- `value_name`: Name der Spalte, die die Werte aus den bestehenden Zeilen erhalten soll
- `value_vars`: Spalten, die die Werte für die `value_name` Spalte enthalten
- `variable_name`: Name der Spalte, die die Spaltennamen aus `value_vars` erhalten soll

In [ ]:
(some_df
    .pivot(values="streams", columns="genre", index="month")
    .melt(id_vars="month", value_name="streams", value_vars=["pop", "rock", "hip-hop"], variable_name="genre")
    .sort("month")
)

Die `null` Werte könnten wir mit `drop_nulls` entfernen.

In [ ]:
(some_df
    .pivot(values="streams", columns="genre", index="month")
    .melt(id_vars="month", value_name="streams", value_vars=["pop", "rock", "hip-hop"], variable_name="genre")
    .sort("month")
    .drop_nulls()
)

## Selektoren + horizontale Ausdrücke

Besonders für Daten im "wide" Format ist es oft hilfreich Operationen auf mehreren Spalten auszuführen, ohne die Spaltennamen konkret angeben zu müssen. Es kann sogar sein, dass die Spaltennamen manchmal bei der Erstellung einer Abfrage noch gar nicht bekannt sind, weil sie erst aus den konkreten Daten entstehen.

Bisher haben wir immer einen einzigen Spaltennamen an `pl.col` übergeben, es gibt aber noch mehr Möglichkeiten:

In [ ]:
# mehrere Spalten mit einem Namen selektieren
df.select(pl.col("rank", "streams").log()).head(2)

In [ ]:
# mehrere Spalten mit einem Datentyp selektieren
df.select(pl.col(pl.Utf8).str.to_lowercase()).head(2)

In [ ]:
# mehrere Spalten mit einem regulären Ausdruck selektieren
df.select(pl.col("^.*rt.*$")).head(2)

Zusätzlich gibt es auch die Möglichkeit alle Spalten zu selektieren.

In [ ]:
df.select(pl.all()).head(2)

Oder auch bestimmte Spalten auszuschließen.

In [ ]:
# alle Spalten, aber nicht "url"
df.select(pl.exclude("url")).head(2)

In [ ]:
# alle Utf8-Spalten, aber nicht "url"
df.select(pl.col(pl.Utf8).exclude("url")).head(2)

Auf so einer Spaltenselektion, die mehr als eine Spalte beinhaltet, können wir auch "horizontale" Berechnungen durchführen. Dafür gibt es die Methoden `pl.horizontal_sum`, `pl.horizontal_min` und `pl.horizontal_max`.

In [ ]:
df.select(pl.sum_horizontal(pl.exclude(pl.Utf8))).head(2)

## Übungen (optional)

### Frage 19

In [ ]:
q19.question()

In [ ]:
q19_df = ...

In [ ]:
q19.check(q19_df)

### Frage 20

In [ ]:
q20.question()

In [ ]:
q12.solution()

In [ ]:
q20_df = ...

In [ ]:
q20.check(df, q20_df)